In [1]:
import pandas as pd
import string
import re
import numpy as np
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer 
import trax
import os
import random as rnd
import trax.fastmath.numpy as np
from trax import layers as tl
from trax import fastmath
import matplotlib as plt

In [2]:
df1=pd.read_csv("Benchmark.csv",encoding='unicode_escape')
df1.head(10)

,ï»¿chatid,user,chatorder,text,ANGRY,HAPPY
0,1,Robert,1,"Hello, is this apple product support?",0.0,0.3
1,1,Bot,2,"Yes, it is. How may I help you?",0.0,0.7
2,1,Robert,3,I have a problem with one of the products that...,0.7,0.0
3,1,Bot,4,I am sorry for that. Can you please tell which...,0.0,0.0
4,1,Robert,5,"It is an iphone 10, it has been a week about n...",1.0,0.0
5,1,Bot,6,I understand the situation and I am glad to as...,0.0,2.0
6,1,Robert,7,My order id is 54569723 and I would like to ha...,0.3,0.0
7,1,Bot,8,"A replacement request has been created, please...",0.0,0.7
8,1,Robert,9,"Thank you for you help, have a good day",0.0,4.3
9,1,Bot,10,"You're welcome, have a great day!",0.0,4.7


In [3]:
positive_text=df1.text[df1.HAPPY>0]
negative_text=df1.text[df1.ANGRY>0]
positive_labels=df1.HAPPY[df1.HAPPY>0]
negative_labels=df1.ANGRY[df1.ANGRY>0]
print(len(positive_text))
print(len(negative_text))
print(len(positive_labels))
print(len(negative_labels))
print(np.shape(negative_labels))



80
73
80
73
(73,)


In [4]:
val_pos = positive_text[64:].values.tolist()
train_pos = positive_text[:64].values.tolist()
val_neg = negative_text[58:].values.tolist()
train_neg = negative_text[:58].values.tolist()
val_lab_pos=positive_labels[64:].values.tolist()
train_lab_pos=positive_labels[:64].values.tolist()
val_lab_neg=negative_labels[58:].values.tolist()
train_lab_neg=negative_labels[:58].values.tolist()

train_x = train_pos + train_neg 
val_x = val_pos + val_neg

In [5]:
train_y = np.array(train_lab_pos+train_lab_neg)
print(train_y)
print(np.shape(train_y))
# Set the labels for the validation set (1 for positive, 0 for negative)
val_y  = np.array(val_lab_pos+val_lab_neg)



[0.3 0.7 2.  0.7 4.3 4.7 3.  0.3 3.  0.7 0.7 0.3 0.3 0.3 3.  3.3 1.  0.7
 1.  0.3 0.7 0.3 1.  0.7 0.3 2.3 1.  4.3 3.3 3.  1.  0.7 0.3 1.  3.  0.3
 4.3 4.7 2.  3.  0.3 0.7 0.7 0.3 0.3 0.3 1.7 0.3 0.7 0.7 0.3 3.3 4.7 4.7
 1.3 0.3 0.7 3.  1.3 3.  1.3 1.  4.3 3.  0.7 1.  0.3 0.3 3.  0.3 4.7 5.
 0.3 0.7 4.3 4.7 0.3 0.7 0.7 0.3 4.  5.  0.3 0.7 0.3 0.3 2.7 3.3 0.3 3.7
 1.3 1.  0.7 0.3 2.7 1.7 1.7 3.7 0.3 0.7 0.3 0.3 0.3 0.3 0.7 0.7 1.  0.3
 1.3 1.3 1.  1.3 1.7 1.7 0.3 3.7 1.  2.3 2.3 1.7 1.7 1.7]
(122,)


In [6]:
def process_text(sent):
    """Process sent function.
    Input:
        sentence: a string containing a sentence
    Output:
        sent_clean: a list of words containing the processed sentence

    """
    #lemmatizer = WordNetLemmatizer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    sent = re.sub(r'\$\w*', '', sent)
    # remove old style retweet text "RT"
    sent = re.sub(r'^RT[\s]+', '', sent)
    # remove hyperlinks
    sent = re.sub(r'https?:\/\/.*[\r\n]*', '', sent)
    # remove hashtags
    # only removing the hash # sign from the word
    sent = re.sub(r'#', '', sent)
    
    #print(word_tokenize(text))
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tokenize=tokenizer.tokenize(sent)
    tokenize=[a.lower() for a in tokenize]
    sent_clean = []
    for d in tokenize:
        if (d not in stopwords_english and  # remove stopwords
                d not in string.punctuation):# remove punctuation
            #lem_sent=lemmatizer.lemmatize(d)
            sent_clean.append(d)
    return sent_clean
    

print(df1['text'].apply(process_text))

0                       [hello, apple, product, support]
1                                       [yes, may, help]
2             [problem, one, products, purchased, store]
3                  [sorry, please, tell, product, issue]
4           [iphone, 10, week, phone, started, overheat]
                             ...                        
192                             [connect, someone, else]
193                                          [apologize]
194                                       [transferring]
195                         [may, phone, number, please]
196    [posting, conversation, online, unbelievable, ...
Name: text, Length: 197, dtype: object


In [7]:
# Build the vocabulary
# Unit Test Note - There is no test set here only train/val

# Include special tokens 
# started with pad, end of line and unk tokens
Vocab = {'__PAD__': 0, '__</e>__': 1, '__UNK__': 2} 

# Note that we build vocab using training data
for sent in train_x: 
    process_sent = process_text(sent)
    for word in process_sent:
        if word not in Vocab: 
            Vocab[word] = len(Vocab)
    
print("Total words in vocab are",len(Vocab))
display(Vocab)

Total words in vocab are 437


{'__PAD__': 0,
 '__</e>__': 1,
 '__UNK__': 2,
 'hello': 3,
 'apple': 4,
 'product': 5,
 'support': 6,
 'yes': 7,
 'may': 8,
 'help': 9,
 'understand': 10,
 'situation': 11,
 'glad': 12,
 'assist': 13,
 'please': 14,
 'share': 15,
 'order': 16,
 'id': 17,
 'let': 18,
 'know': 19,
 'would': 20,
 'like': 21,
 'return': 22,
 'replacement': 23,
 'request': 24,
 'created': 25,
 'carry': 26,
 'device': 27,
 'nearest': 28,
 'store': 29,
 'representatives': 30,
 'verify': 31,
 'replace': 32,
 'thank': 33,
 'good': 34,
 'day': 35,
 'welcome': 36,
 'great': 37,
 'today': 38,
 'bank': 39,
 'wealth': 40,
 'name': 41,
 'heather': 42,
 'remaining': 43,
 'money': 44,
 'account': 45,
 'iâ': 46,
 '\x80': 47,
 '\x99': 48,
 'get': 49,
 'number': 50,
 'sure': 51,
 'tracy': 52,
 'q': 53,
 'randall': 54,
 '8051-7845-': 55,
 '3895-0611': 56,
 'check': 57,
 'ok': 58,
 'last': 59,
 'four': 60,
 'numbers': 61,
 'social': 62,
 'security': 63,
 'still': 64,
 '84': 65,
 'thousand': 66,
 '65': 67,
 'cents': 68,
 'an

In [8]:
# UNQ_C1 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: sent_to_tensor
def sent_to_tensor(sent, vocab_dict, unk_token='__UNK__', verbose=False):
    '''
    Input: 
        sent - A string containing a sentence
        vocab_dict - The words dictionary
        unk_token - The special string for unknown tokens
        verbose - Print info durign runtime
    Output:
        tensor_l - A python list with
        
    '''  
    
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    # Process the sentence into a list of words
    # where only important words are kept (stop words removed)
    word_l = process_text(sent)
    
    if verbose:
        print("List of words from the processed sentence:")
        print(word_l)
        
    # Initialize the list that will contain the unique integer IDs of each word
    tensor_l = []
    
    # Get the unique integer ID of the __UNK__ token
    unk_ID = Vocab.get('__UNK__')
    
    if verbose:
        print(f"The unique integer ID for the unk_token is {unk_ID}")
        
    # for each word in the list:
    for word in word_l:
        
        # Get the unique integer ID.
        # If the word doesn't exist in the vocab dictionary,
        # use the unique ID for __UNK__ instead.
        word_ID = vocab_dict.get(word,unk_ID)
    ### END CODE HERE ###
        
        # Append the unique integer ID to the tensor list.
        tensor_l.append(word_ID) 
    
    return tensor_l

In [9]:
print("Actual sentence is\n", val_pos[1])
print("\nTensor of sentence:\n", sent_to_tensor(val_pos[1], vocab_dict=Vocab))

Actual sentence is
 Yes I can help you. Could you please share me your order id 

Tensor of sentence:
 [7, 9, 71, 14, 15, 16, 17]


In [10]:
# test sent_to_tensor

def test_sent_to_tensor():
    test_cases = [
        
        {
            "name":"simple_test_check",
            "input": [val_pos[1], Vocab],
            "expected":[7, 9, 71, 14, 15, 16, 17],
            "error":"The function gives bad output for val_pos[1]. Test failed"
        },
        {
            "name":"datatype_check",
            "input":[val_pos[1], Vocab],
            "expected":type([]),
            "error":"Datatype mismatch. Need only list not np.array"
        },
        {
            "name":"without_unk_check",
            "input":[val_pos[1], Vocab],
            "expected":6,
            "error":"Unk word check not done- Please check if you included mapping for unknown word"
        }
    ]
    count = 0
    for test_case in test_cases:
        
        try:
            if test_case['name'] == "simple_test_check":
                assert test_case["expected"] == sent_to_tensor(*test_case['input'])
                count += 1
            if test_case['name'] == "datatype_check":
                assert isinstance(sent_to_tensor(*test_case['input']), test_case["expected"])
                count += 1
            if test_case['name'] == "without_unk_check":
                assert None not in sent_to_tensor(*test_case['input'])
                count += 1
                
            
            
        except:
            print(test_case['error'])
    if count == 3:
        print("\033[92m All tests passed")
    else:
        print(count," Tests passed out of 3")
test_sent_to_tensor()  

 All tests passed


In [11]:
def data_generator(data_pos, data_neg, batch_size, loop, vocab_dict, shuffle=False):
    '''
    Input: 
        data_pos - Set of posstive examples
        data_neg - Set of negative examples
        batch_size - number of samples per batch. Must be even
        loop - True or False
        vocab_dict - The words dictionary
        shuffle - Shuffle the data order
    Yield:
        inputs - Subset of positive and negative examples
        targets - The corresponding labels for the subset
        example_weights - An array specifying the importance of each example
        
    '''     
### START GIVEN CODE ###
    # make sure the batch size is an even number
    # to allow an equal number of positive and negative samples
    assert batch_size % 2 == 0
    
    # Number of positive examples in each batch is half of the batch size
    # same with number of negative examples in each batch
    n_to_take = batch_size // 2
    
    
    # Use pos_index to walk through the data_pos array
    # same with neg_index and data_neg
    pos_index = 0
    neg_index = 0
    
    len_data_pos = len(data_pos)
    len_data_neg = len(data_neg)
   
    
    # Get and array with the data indexes
    pos_index_lines = list(range(len_data_pos))
    neg_index_lines = list(range(len_data_neg))
    
    # shuffle lines if shuffle is set to True
    if shuffle:
        rnd.shuffle(pos_index_lines)
        rnd.shuffle(neg_index_lines)
        
    stop = False
    
    # Loop indefinitely
    while not stop:  
        
        # create a batch with positive and negative examples
        batch = []
        
        # First part: Pack n_to_take positive examples
        
        # Start from pos_index and increment i up to n_to_take
        for i in range(n_to_take):
                    
            # If the positive index goes past the positive dataset lenght,
            if pos_index >= len_data_pos: 
                
                # If loop is set to False, break once we reach the end of the dataset
                if not loop:
                    stop = True;
                    break;
                
                # If user wants to keep re-using the data, reset the index
                pos_index = 0
                
                if shuffle:
                    # Shuffle the index of the positive sample
                    rnd.shuffle(pos_index_lines)
                    
            # get the sentence as pos_index
            sent = data_pos[pos_index_lines[pos_index]]
           
            # convert the sentence into tensors of integers representing the processed words
            tensor = sent_to_tensor(sent, vocab_dict)
           
            # append the tensor to the batch list
            batch.append(tensor)
            
            # Increment pos_index by one
            pos_index = pos_index + 1

### END GIVEN CODE ###
            
### START CODE HERE (Replace instances of 'None' with your code) ###

        # Second part: Pack n_to_take negative examples
    
        # Using the same batch list, start from neg_index and increment i up to n_to_take
        for i in range(n_to_take):
            
            # If the negative index goes past the negative dataset length,
            if neg_index >=len_data_neg:
                
                # If loop is set to False, break once we reach the end of the dataset
                if not loop:
                    stop = True;
                    break;
                    
                # If user wants to keep re-using the data, reset the index
                neg_index = 0
                
                if shuffle:
                    # Shuffle the index of the negative sample
                    rnd.shuffle(neg_index_lines)
            # get the sntence as neg_index
            sent = data_neg[neg_index_lines[neg_index]]
            
            # convert the sentence into tensors of integers representing the processed words
            tensor = sent_to_tensor(sent,vocab_dict)
            
            # append the tensor to the batch list
            batch.append(tensor)
            
            # Increment neg_index by one
            neg_index = neg_index+1
            
### END CODE HERE ###        

### START GIVEN CODE ###
        if stop:
            break;

        # Update the start index for positive data 
        # so that it's n_to_take positions after the current pos_index
        pos_index += n_to_take
        
        # Update the start index for negative data 
        # so that it's n_to_take positions after the current neg_index
        neg_index += n_to_take
        
        # Get the max sentence length (the length of the longest sentence) 
        # (you will pad all shorter sentences to have this length)
        max_len = max([len(t) for t in batch]) 
        
        
        # Initialize the input_l, which will d 
        # store the padded versions of the tensors
        tensor_pad_l = []
        # Pad shorter sentences with zeros
        for tensor in batch:
### END GIVEN CODE ###

### START CODE HERE (Replace instances of 'None' with your code) ###
            # Get the number of positions to pad for this tensor so that it will be max_len long
            n_pad = max_len - len(tensor)
            
            # Generate a list of zeros, with length n_pad
            pad_l = [0] * n_pad
            
            # concatenate the tensor and the list of padded zeros
            tensor_pad = tensor + pad_l 
            
            # append the padded tensor to the list of padded tensors
            tensor_pad_l.append(tensor_pad)

        # convert the list of padded tensors to a numpy array
        # and store this as the model inputs
        inputs = np.array(tensor_pad_l)
  
        # Generate the list of targets for the positive examples (a list of ones)
        # The length is the number of positive examples in the batch
        target_pos =  [1]*n_to_take
        
        # Generate the list of targets for the negative examples (a list of zeros)
        # The length is the number of negative examples in the batch
        target_neg = [0]*n_to_take
        
        # Concatenate the positve and negative targets
        target_l = target_pos+target_neg
        
        # Convert the target list into a numpy array
        targets = np.array(target_l)
        
        # Example weights: Treat all examples equally importantly.It should return an np.array. Hint: Use np.ones_like()
        example_weights = np.ones_like(targets)
        

### END CODE HERE ###

### GIVEN CODE ###
        # note we use yield and not return
        yield inputs, targets, example_weights
        

In [12]:
# Set the random number generator for the shuffle procedure
rnd.seed(30) 

# Create the training data generator
def train_generator(batch_size, shuffle = False):
    return data_generator(train_pos, train_neg, batch_size, True, Vocab, shuffle)

# Create the validation data generator
def val_generator(batch_size, shuffle = False):
    return data_generator(val_pos, val_neg, batch_size, True, Vocab, shuffle)

# Create the validation data generator
def test_generator(batch_size, shuffle = False):
    return data_generator(val_pos, val_neg, batch_size, False, Vocab, shuffle)

# Get a batch from the train_generator and inspect.
inputs, targets, example_weights = next(train_generator(4, shuffle=True))

# this will print a list of 4 tensors padded with zeros
print(f'Inputs: {inputs}')
print(f'Targets: {targets}')
print(f'Example Weights: {example_weights}')

Inputs: [[ 92  93  94  95  96   9  42   0   0   0   0   0   0   0   0]
 [186 187 188 189 190 191 192 193 194 195 196 197  64  92 184]
 [434   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  3   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
Targets: [1 1 0 0]
Example Weights: [1 1 1 1]


In [13]:
# Test the train_generator

# Create a data generator for training data,
# which produces batches of size 4 (for tensors and their respective targets)
tmp_data_gen = train_generator(batch_size = 4)

# Call the data generator to get one batch and its targets
tmp_inputs, tmp_targets, tmp_example_weights = next(tmp_data_gen)

print(f"The inputs shape is {tmp_inputs.shape}")
print(f"The targets shape is {tmp_targets.shape}")
print(f"The example weights shape is {tmp_example_weights.shape}")

for i,t in enumerate(tmp_inputs):
    print(f"input tensor: {t}; target {tmp_targets[i]}; example weights {tmp_example_weights[i]}")

The inputs shape is (4, 6)
The targets shape is (4,)
The example weights shape is (4,)
input tensor: [3 4 5 6 0 0]; target 1; example weights 1
input tensor: [7 8 9 0 0 0]; target 1; example weights 1
input tensor: [244 107 255 256  29   0]; target 0; example weights 1
input tensor: [257 258 259  78 260 261]; target 0; example weights 1


In [14]:
class Layer(object):
    """Base class for layers."""
    def __init__(self):
        self.weights = None

    def forward(self, x):
        raise NotImplementedError
  
    def init_weights_and_state(self, input_signature, random_key):
        pass

    def init(self, input_signature, random_key):
        self.init_weights_and_state(input_signature, random_key)
        return self.weights
    
    def __call__(self, x):
        return self.forward(x)

In [15]:
class Relu(Layer):
    """Relu activation function implementation"""
    def forward(self, x):
        '''
        Input: 
            - x (a numpy array): the input
        Output:
            - activation (numpy array): all positive or 0 version of x
        '''
        ### START CODE HERE (Replace instances of 'None' with your code) ###
        
        activation = np.maximum(x,0)

        ### END CODE HERE ###
        
        return activation

In [16]:
# use the fastmath module within trax
from trax import fastmath

# use the numpy module from trax
np = fastmath.numpy

# use the fastmath.random module from trax
random = fastmath.random

In [17]:
class Dense(Layer):
    """
    A dense (fully-connected) layer.
    """

    # __init__ is implemented for you
    def __init__(self, n_units, init_stdev=0.1):
        
        # Set the number of units in this layer
        self._n_units = n_units
        self._init_stdev = init_stdev

    # Please implement 'forward()'
    def forward(self, x):

### START CODE HERE (Replace instances of 'None' with your code) ###

        # Matrix multiply x and the weight matrix
        dense = np.dot(x,self.weights) 
        
### END CODE HERE ###
        return dense

    # init_weights
    def init_weights_and_state(self, input_signature, random_key):
        
### START CODE HERE (Replace instances of 'None' with your code) ###
        # The input_signature has a .shape attribute that gives the shape as a tuple
        input_shape = (input_signature.shape[-1], self._n_units)

        # Generate the weight matrix from a normal distribution, 
        # and standard deviation of 'stdev'        
        w = trax.fastmath.random.normal(key=random_key, shape=input_shape) * self._init_stdev
        
### END CODE HERE ###     
        self.weights = w
        return self.weights

In [18]:
# Testing your Dense layer 
dense_layer = Dense(n_units=10)  #sets  number of units in dense layer (n_units=10)
random_key = random.get_prng(seed=0)  # sets random seed
z = np.array([[2.0, 7.0, 25.0]]) # input array 

dense_layer.init(z, random_key)
print("Weights are\n ",dense_layer.weights) #Returns randomly generated weights
print("Foward function output is ", dense_layer(z)) # Returns multiplied values of units and weights

Weights are
  [[-0.02837108  0.09368162 -0.10050076  0.14165013  0.10543301  0.09108126
  -0.04265672  0.0986188  -0.05575325  0.00153249]
 [-0.20785688  0.0554837   0.09142365  0.05744595  0.07227863  0.01210617
  -0.03237354  0.16234995  0.02450038 -0.13809784]
 [-0.06111237  0.01403724  0.08410042 -0.1094358  -0.10775021 -0.11396459
  -0.05933381 -0.01557652 -0.03832145 -0.11144515]]
Foward function output is  [[-3.0395496   0.9266802   2.5414743  -2.050473   -1.9769388  -2.582209
  -1.7952735   0.94427425 -0.8980402  -3.7497487 ]]


In [19]:
# GRADED FUNCTION: classifier
def classifier(vocab_size=len(Vocab), embedding_dim=256, output_dim=2, mode='train'):
        
### START CODE HERE (Replace instances of 'None' with your code) ###
    # create embedding layer
    embed_layer = tl.Embedding(
        vocab_size=vocab_size, # Size of the vocabulary
        d_feature=embedding_dim)  # Embedding dimension
    
    # Create a mean layer, to create an "average" word embedding
    mean_layer = tl.Mean(axis=1)
    
    # Create a dense layer, one unit for each output
    dense_output_layer = tl.Dense(n_units = output_dim)

    
    # Create the log softmax layer (no parameters needed)
    softmax_layer = tl.Softmax()
    
    # Use tl.Serial to combine all layers
    # and create the classifier
    # of type trax.layers.combinators.Serial
    model = tl.Serial(
      embed_layer, # embedding layer
      mean_layer, # mean layer
      dense_output_layer, # dense output layer 
      softmax_layer # log softmax layer
    )
### END CODE HERE ###     
    
    # return the model of type
    return model

In [20]:
tmp_model = classifier()

In [21]:
print(type(tmp_model))
display(tmp_model)

<class 'trax.layers.combinators.Serial'>


Serial[
  Embedding_437_256
  Mean
  Dense_2
  Softmax
]

In [22]:
from trax.supervised import training

batch_size = 16
rnd.seed(271)

train_task = training.TrainTask(
    labeled_data=train_generator(batch_size=batch_size, shuffle=True),
    loss_layer=tl.WeightedCategoryCrossEntropy(),
    optimizer=trax.optimizers.Adam(0.01),
    n_steps_per_checkpoint=20,
)

eval_task = training.EvalTask(
    labeled_data=val_generator(batch_size=batch_size, shuffle=True),
    metrics=[tl.WeightedCategoryCrossEntropy(), tl.WeightedCategoryAccuracy()],
)

model = classifier()

In [23]:
output_dir = '~/model/'
output_dir_expand = os.path.expanduser(output_dir)
print(output_dir_expand)

/Users/lexroverts/model/


In [24]:
# GRADED FUNCTION: train_model
def train_model(classifier, train_task, eval_task, n_steps, output_dir):
    '''
    Input: 
        classifier - the model you are building
        train_task - Training task
        eval_task - Evaluation task
        n_steps - the evaluation steps
        output_dir - folder to save your files
    Output:
        trainer -  trax trainer
    '''
### START CODE HERE (Replace instances of 'None' with your code) ###
    training_loop = training.Loop(
                                classifier, # The learning model
                                train_task, # The training task
                                eval_tasks = [eval_task], # The evaluation task
                                output_dir = output_dir) # The output directory

    training_loop.run(n_steps = n_steps)
### END CODE HERE ###

    # Return the training_loop, since it has the model.
    return training_loop

In [25]:
training_loop = train_model(model, train_task, eval_task,100, output_dir_expand)


Step   1340: Ran 20 train steps in 8.00 secs
Step   1340: train WeightedCategoryCrossEntropy |  0.33322650
Step   1340: eval  WeightedCategoryCrossEntropy |  0.68531901
Step   1340: eval      WeightedCategoryAccuracy |  0.62500000


In [26]:
# GRADED FUNCTION: compute_accuracy
def compute_accuracy(preds, y, y_weights):
    """
    Input: 
        preds: a tensor of shape (dim_batch, output_dim) 
        y: a tensor of shape (dim_batch,) with the true labels
        y_weights: a n.ndarray with the a weight for each example
    Output: 
        accuracy: a float between 0-1 
        weighted_num_correct (np.float32): Sum of the weighted correct predictions
        sum_weights (np.float32): Sum of the weights
    """
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    # Create an array of booleans, 
    # True if the probability of positive sentiment is greater than
    # the probability of negative sentiment
    # else False
    is_pos =  preds[:,1] > preds[:,0]

    # convert the array of booleans into an array of np.int32
    is_pos_int = is_pos.astype(np.int32)
    
    # compare the array of predictions (as int32) with the target (labels) of type int32
    correct = np.equal(is_pos_int, y)

    # Count the sum of the weights.
    sum_weights = np.sum(y_weights)
    
    # convert the array of correct predictions (boolean) into an arrayof np.float32
    correct_float = correct.astype(np.float32)
    
    # Multiply each prediction with its corresponding weight.
    weighted_correct_float = correct_float*sum_weights

    # Sum up the weighted correct predictions (of type np.float32), to go in the
    # denominator.
    weighted_num_correct = np.sum(correct_float)
 
    # Divide the number of weighted correct predictions by the sum of the
    # weights.
    accuracy =   weighted_num_correct/sum_weights

    ### END CODE HERE ###
    return accuracy, weighted_num_correct, sum_weights

In [27]:
# test your function (64)
tmp_val_generator = val_generator(10)

# get one batch
tmp_batch = next(tmp_val_generator)

# Position 0 has the model inputs (sentence as tensors)
# position 1 has the targets (the actual labels)
tmp_inputs, tmp_targets, tmp_example_weights = tmp_batch

# feed the sentence tensors into the model to get a prediction
tmp_pred = training_loop.eval_model(tmp_inputs)

tmp_acc, tmp_num_correct, tmp_num_predictions = compute_accuracy(preds=tmp_pred, y=tmp_targets, y_weights=tmp_example_weights)

print(f"Model's prediction accuracy on a single training batch is: {100 * tmp_acc}%")
print(f"Weighted number of correct predictions {tmp_num_correct}; weighted number of total observations predicted {tmp_num_predictions}")

Model's prediction accuracy on a single training batch is: 80.0%
Weighted number of correct predictions 8.0; weighted number of total observations predicted 10


In [28]:
for sent in train_x:
    inputs = np.array(sent_to_tensor(sent, vocab_dict=Vocab))
    inputs = inputs[None, :]
    predictions=model(inputs)
    print(f'example input_str: {sent}')
    print(f'Model returned sentiment probabilities: {predictions.argmax(axis=1)}')

example input_str: Hello, is this apple product support?
Model returned sentiment probabilities: [1]
example input_str: Yes, it is. How may I help you?
Model returned sentiment probabilities: [1]
example input_str: I understand the situation and I am glad to assist you with it, can you please share the order id and let me know if you would like to have a return or replacement for it.
Model returned sentiment probabilities: [1]
example input_str: A replacement request has been created, please carry the device to the nearest store and our representatives will verify and replace the product.
Model returned sentiment probabilities: [1]
example input_str: Thank you for you help, have a good day
Model returned sentiment probabilities: [1]
example input_str: You're welcome, have a great day!
Model returned sentiment probabilities: [1]
example input_str: It is a good day today at Bank of Wealth. My name is Heather. How can I he you?
Model returned sentiment probabilities: [1]
example input_str

LayerError: Exception passing through layer Serial (in pure_fn):
  layer created in file [...]/<ipython-input-19-118126b81272>, line 27
  layer input shapes: ShapeDtype{shape:(1, 0), dtype:float32}

  File [...]/trax/layers/combinators.py, line 88, in forward
    outputs, s = layer.pure_fn(inputs, w, s, rng, use_cache=True)

LayerError: Exception passing through layer Embedding_437_256 (in pure_fn):
  layer created in file [...]/<ipython-input-19-118126b81272>, line 8
  layer input shapes: ShapeDtype{shape:(1, 0), dtype:float32}

  File [...]/trax/layers/assert_shape.py, line 122, in forward_wrapper
    y = forward(self, x, *args, **kwargs)

  File [...]/trax/layers/core.py, line 181, in forward
    embedded = jnp.take(self.weights, x, axis=0)

  File [...]/_src/numpy/lax_numpy.py, line 4078, in take
    slice_sizes=tuple(slice_sizes))

  File [...]/_src/lax/lax.py, line 874, in gather
    slice_sizes=canonicalize_shape(slice_sizes))

  File [...]/site-packages/jax/core.py, line 282, i